In [2]:
# notebooks/EDA/eda.ipynb

# --- Настройка окружения ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Загрузка данных ---
sessions = pd.read_pickle('../../data/processed_data/processed_sessions.pkl')
hits = pd.read_pickle('../../data/processed_data/processed_hits.pkl')

print(f"✅ sessions.shape = {sessions.shape}")
print(f"✅ hits.shape = {hits.shape}")

# --- 1. Первичный обзор ---
print("\n--- Sessions Info ---")
print(sessions.info())
print("\n--- Hits Info ---")
print(hits.info())

print("\n--- Sessions Target Distribution ---")
print(sessions['target'].value_counts(normalize=True))

# --- 2. Распределение целевой переменной ---
plt.figure()
sessions['target'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Target Distribution')
plt.xlabel('Target (0/1)')
plt.ylabel('Count')
plt.show()

# --- 3. Числовые признаки ---
numeric_features = ['visit_number', 'screen_width', 'screen_height']

print("\n--- Numeric Features Description ---")
print(sessions[numeric_features].describe())

for col in numeric_features:
    plt.figure()
    sns.histplot(sessions[col].dropna(), kde=True, bins=30)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

# --- 4. Категориальные признаки ---
categorical_features = [
    'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword',
    'device_category', 'device_os', 'device_brand', 'device_model',
    'device_browser', 'geo_country', 'geo_city'
]

for col in categorical_features:
    if sessions[col].nunique() < 30:  # чтобы не строить слишком много баров
        plt.figure()
        sns.countplot(data=sessions, y=col, order=sessions[col].value_counts().index)
        plt.title(f'Distribution of {col}')
        plt.xlabel('Count')
        plt.ylabel(col)
        plt.show()

# --- 5. Корреляция числовых признаков с таргетом ---
corr_matrix = sessions[numeric_features + ['target']].corr()

plt.figure()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

# --- 6. Поведение пользователей ---
# Сколько событий на сессию?
events_per_session = hits.groupby('session_id').size().rename('n_events')

# Объединяем
sessions = sessions.join(events_per_session, on='session_id').fillna(0)

# Распределение количества событий
plt.figure()
sns.histplot(sessions['n_events'], bins=50, kde=True)
plt.xlim(0, 100)
plt.title('Number of Events per Session')
plt.xlabel('Number of Events')
plt.ylabel('Frequency')
plt.show()

# Boxplot по таргету
plt.figure()
sns.boxplot(x='target', y='n_events', data=sessions)
plt.title('Events per Session by Target')
plt.xlabel('Target')
plt.ylabel('Number of Events')
plt.ylim(0, 100)
plt.show()

# --- 7. ТОП страничек и рефереров ---
print("\n--- Top Pages ---")
print(hits['hit_page_path'].value_counts().head(10))

plt.figure()
hits['hit_page_path'].value_counts().head(10).plot(kind='barh')
plt.title('Top 10 Pages')
plt.xlabel('Hits')
plt.ylabel('Page Path')
plt.show()

print("\n--- Top Referers ---")
print(hits['hit_referer'].value_counts().head(10))

plt.figure()
hits['hit_referer'].value_counts().head(10).plot(kind='barh', color='orange')
plt.title('Top 10 Referers')
plt.xlabel('Hits')
plt.ylabel('Referer')
plt.show()

# --- 8. Анализ действий (event_action) ---
print("\n--- Top Event Actions ---")
print(hits['event_action'].value_counts().head(10))

plt.figure()
hits['event_action'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Event Actions')
plt.xlabel('Event Action')
plt.ylabel('Count')
plt.show()

# --- 9. Конечный вывод ---
print("\n--- Summary ---")
print(f"Всего сессий: {sessions.shape[0]}")
print(f"Всего событий: {hits.shape[0]}")
print(f"Конверсия (target=1): {sessions['target'].mean():.4f}")
print(f"Среднее число событий на сессию: {sessions['n_events'].mean():.2f}")

print("\n📊 Базовый EDA завершён.")


✅ sessions.shape = (1860042, 21)
✅ hits.shape = (15726470, 12)

--- Sessions Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860042 entries, 0 to 1860041
Data columns (total 21 columns):
 #   Column           Dtype         
---  ------           -----         
 0   session_id       object        
 1   client_id        object        
 2   visit_date       datetime64[ns]
 3   visit_time       category      
 4   visit_number     int16         
 5   utm_source       category      
 6   utm_medium       category      
 7   utm_campaign     category      
 8   utm_adcontent    category      
 9   utm_keyword      category      
 10  device_category  category      
 11  device_os        category      
 12  device_brand     category      
 13  device_model     category      
 14  device_browser   category      
 15  geo_country      category      
 16  geo_city         category      
 17  session_start    datetime64[ns]
 18  screen_width     int16         
 19  screen_height    in

KeyError: 'target'